# Deutsch Jozsa Algorithm

Import needed modules

In [1]:
""" This module implements Deutch_Jozsa algorithm """
from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import Qubit, IntQubit, matrix_to_qubit
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.gate import HadamardGate
from sympy.physics.quantum.represent import represent

from oracle import oracle
from util.util import hn

from sympy import sqrt

## The problem:
This is a generalization of Deutsch algorithm consider `f {0,1}^n  -> f {0, 1}^n` that is not known determine if f is balanced or constant where f is balanced if exactly half of the input go to zero ( and the other half to one) constant if all input go to zero OR all got to one

## The Solution:
```                         
                     +-----------------+
  |x>  |0>-/n--H*n---|                 |--/n--H*n---M |x>
                     |       U_f       |
  |y>  |1>---H-------|                 |------------  |y XOR f(x)>
                     +-----------------+
```
```
Constant zero -> +|0*n>*|->
Constant  one -> -|0*n>*|->
Balanced 0->1 ->  NOT (+|0*n>*|->) and NOT (-|0*n>*|->)
```

In [2]:
def deutsch_jozsa(f, n):
    """
    Run  Deutsch Jozsa algorithm and return |x>*|y XOR f(x)>
    :param func f: oracle function
    :param int n: string length

    """
    x = qapply(hn(n) * Qubit('0' * n))
    print(f"|x>: {x}")
    y = qapply(HadamardGate(0) * Qubit(1))
    print(f"|y>: {y}")

    # calculate the Tensor Product of the inputs
    xy = TensorProduct(x, y)
    xy = matrix_to_qubit(represent(xy))
    print(f"|xy>: {xy}")

    # apply oracle
    r = oracle(x, y, f)

    # apply H*n on top Qubits
    r = qapply(hn(n, start=1) * r)
    return r

## Tests

Test Const zero function

In [3]:
def test_0():

    def f(x, *args):
        return 0

    n = 3
    r = deutsch_jozsa(f, n)
    print(f'\nResult const zero function: {r}')

    truth = TensorProduct(Qubit('0' * n), 1 / sqrt(2) * (Qubit(0) - Qubit(1)))
    truth = matrix_to_qubit(represent((truth)))
    assert truth == r


test_0()

|x>: sqrt(2)*|000>/4 + sqrt(2)*|001>/4 + sqrt(2)*|010>/4 + sqrt(2)*|011>/4 + sqrt(2)*|100>/4 + sqrt(2)*|101>/4 + sqrt(2)*|110>/4 + sqrt(2)*|111>/4
|y>: sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>: |0000>/4 - |0001>/4 + |0010>/4 - |0011>/4 + |0100>/4 - |0101>/4 + |0110>/4 - |0111>/4 + |1000>/4 - |1001>/4 + |1010>/4 - |1011>/4 + |1100>/4 - |1101>/4 + |1110>/4 - |1111>/4

Result const zero function: sqrt(2)*|0000>/2 - sqrt(2)*|0001>/2


Test Const one function

In [ ]:
def test_1():

    def f(x, *args):
        return 1

    n = 3
    r = deutsch_jozsa(f, n)
    print(f'\nResult const one function: {r}')

    truth = TensorProduct(-1 * Qubit('0' * n),
                          1 / sqrt(2) * (Qubit(0) - Qubit(1)))
    truth = matrix_to_qubit(represent(truth))
    assert truth == r


test_1()

|x>: sqrt(2)*|000>/4 + sqrt(2)*|001>/4 + sqrt(2)*|010>/4 + sqrt(2)*|011>/4 + sqrt(2)*|100>/4 + sqrt(2)*|101>/4 + sqrt(2)*|110>/4 + sqrt(2)*|111>/4
|y>: sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>: |0000>/4 - |0001>/4 + |0010>/4 - |0011>/4 + |0100>/4 - |0101>/4 + |0110>/4 - |0111>/4 + |1000>/4 - |1001>/4 + |1010>/4 - |1011>/4 + |1100>/4 - |1101>/4 + |1110>/4 - |1111>/4

Result const one function: -sqrt(2)*|0000>/2 + sqrt(2)*|0001>/2


Test balanced function

In [ ]:
def test_2():

    def f(x, *args):
        q = IntQubit(*x)
        if q.as_int() % 2 == 0:
            return 1
        else:
            return 0

    n = 3
    r = deutsch_jozsa(f, n)
    print(f'\nResult balanced zero function: {r}')
    truth_0 = TensorProduct(Qubit('0' * n),
                            1 / sqrt(2) * (Qubit(0) - Qubit(1)))
    truth_0 = matrix_to_qubit(represent((truth_0)))
    truth_1 = TensorProduct(-1 * Qubit('0' * n),
                            1 / sqrt(2) * (Qubit(0) - Qubit(1)))
    truth_1 = matrix_to_qubit(represent(truth_1))

    assert r != truth_0 and r != truth_1


test_2()

|x>: sqrt(2)*|000>/4 + sqrt(2)*|001>/4 + sqrt(2)*|010>/4 + sqrt(2)*|011>/4 + sqrt(2)*|100>/4 + sqrt(2)*|101>/4 + sqrt(2)*|110>/4 + sqrt(2)*|111>/4
|y>: sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>: |0000>/4 - |0001>/4 + |0010>/4 - |0011>/4 + |0100>/4 - |0101>/4 + |0110>/4 - |0111>/4 + |1000>/4 - |1001>/4 + |1010>/4 - |1011>/4 + |1100>/4 - |1101>/4 + |1110>/4 - |1111>/4

Result balanced zero function: -sqrt(2)*|0010>/2 + sqrt(2)*|0011>/2
